In [1]:
import pandas as pd
import numpy as np

In [2]:
#Reading Datasets
books = pd.read_csv('BX-Books.csv',encoding='latin-1')
ratings = pd.read_csv('BX-Book-Ratings.csv',encoding='latin-1')

D:\softwares\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
books.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [4]:
print(books.shape)
print(books.dtypes)

(271377, 5)
isbn                   object
book_title             object
book_author            object
year_of_publication    object
publisher              object
dtype: object


In [5]:
ratings.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [6]:
print(ratings.shape)
print(ratings.dtypes)
print(ratings.isnull().sum())

(1048575, 3)
user_id     int64
isbn       object
rating      int64
dtype: object
user_id    0
isbn       0
rating     0
dtype: int64


In [7]:
#Keeping only the Users with atleast 200 Book Ratings
counts1 = ratings['user_id'].value_counts()
counts1

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
104999        1
105002        1
105008        1
105014        1
123969        1
Name: user_id, Length: 95513, dtype: int64

In [8]:
ratings = ratings[ratings['user_id'].isin(counts1[counts1 >= 200].index)] #Removing Users with less than 200 Book Ratings
print(ratings.shape)
ratings.head()

(483728, 3)


,user_id,isbn,rating
1456,277427,002542730X,10
1457,277427,26217457,0
1458,277427,003008685X,8
1459,277427,30615321,0
1460,277427,60002050,0


In [9]:
counts = ratings['rating'].value_counts(sort = False)
counts

10     26130
0     359642
8      29215
7      19592
9      21966
6       8349
5      15078
4       1632
3       1050
2        561
1        513
Name: rating, dtype: int64

In [10]:
#Combining Rating Dataset with Books dataset on Basis og ISBN number of Book
combine_book_rating = pd.merge(ratings, books, on='isbn')  
combine_book_rating.head()

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc


In [11]:
#Creating new Dataframe with Book Title and number of Ratings on it.
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['book_title'])
book_ratingCount = (combine_book_rating.
     groupby(by = ['book_title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
    )
book_ratingCount.head()

,book_title,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [12]:
#Combining the "combine_book_rating" and "book_ratingCount"
#So that we get no. of rating, actual rating and title of book in one single Dataframe.
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'book_title', right_on = 'book_title', how = 'left')
print(rating_with_totalRatingCount.shape)
rating_with_totalRatingCount.head()

(447648, 8)


,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher,totalRatingCount
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74


In [13]:
#Eliminating Users with less than 50 totalRatingCount
popularity_threshold = 50
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
print(rating_popular_book.shape)
rating_popular_book.head()

(50914, 8)


,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher,totalRatingCount
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74


In [14]:
unique_df = rating_popular_book.drop('user_id', axis='columns')
unique_df = unique_df.drop('rating', axis='columns')
unique_df.drop_duplicates('book_title')

,isbn,book_title,book_author,year_of_publication,publisher,totalRatingCount
0,002542730X,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
180,60930535,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,123
315,60934417,Bel Canto: A Novel,Ann Patchett,2002,Perennial,101
421,61009059,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,99
575,006440188X,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,72
...,...,...,...,...,...,...
170402,451186362,Rose Madder,Stephen King,2004,Signet Book,74
175564,770427758,Drums of Autumn,Diana Gabaldon,1997,Seal Press (WA),52
182260,671867156,Pretend You Don't See Her,Mary Higgins Clark,1998,Pocket,95
185956,553287397,Dark Angel,Sally Beauman,1991,Bantam Books,50


In [15]:
#Creating Pivot Table: 
rating_popular_book = rating_popular_book.drop_duplicates(['user_id', 'book_title'])
rating_popular_book_pivot = rating_popular_book.pivot(index = 'book_title', columns = 'user_id', values = 'rating')
print(rating_popular_book_pivot.shape)
rating_popular_book_pivot.head()

(628, 809)


user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,249628,249862,249894,250184,250405,250764,277427,277478,277639,278418
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
rating_popular_book_pivot = rating_popular_book_pivot.fillna(0) #Replacing "NaN" values with "0"
rating_popular_book_pivot.head()

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,249628,249862,249894,250184,250405,250764,277427,277478,277639,278418
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
for i in range(628):
    print(rating_popular_book_pivot.index[i])

1984
1st to Die: A Novel
2nd Chance
4 Blondes
84 Charing Cross Road
A Bend in the Road
A Case of Need
A Child Called \It\": One Child's Courage to Survive"
A Civil Action
A Day Late and a Dollar Short
A Fine Balance
A Heartbreaking Work of Staggering Genius
A Is for Alibi (Kinsey Millhone Mysteries (Paperback))
A Lesson Before Dying (Vintage Contemporaries (Paperback))
A Man in Full
A Map of the World
A Painted House
A Patchwork Planet
A Prayer for Owen Meany
A Thin Dark Line (Mysteries &amp; Horror)
A Thousand Acres (Ballantine Reader's Circle)
A Time to Kill
A Virtuous Woman (Oprah's Book Club (Paperback))
A Walk to Remember
A Widow for One Year
A Wrinkle In Time
A Wrinkle in Time
ANGELA'S ASHES
About a Boy
Absolute Power
Accident
Airframe
All Around the Town
All I Really Need to Know
All That Remains (Kay Scarpetta Mysteries (Paperback))
All the Pretty Horses (The Border Trilogy, Vol 1)
Along Came a Spider (Alex Cross Novels)
American Gods
Angela's Ashes (MMP) : A Memoir
Angela's As

In [18]:
import pickle
file = "book_pivot.pkl"
fileobj = open(file, 'wb')
pickle.dump(rating_popular_book_pivot, fileobj)
fileobj.close()

In [19]:
file = "book_pivot.pkl"
fileobj = open(file, 'rb')
pivot = pickle.load(fileobj)

In [20]:
from scipy.sparse import csr_matrix
rating_popular_book_matrix = csr_matrix(rating_popular_book_pivot.values) #Creating matrix out of pivot table which can be given to knn
#rating_popular_book_matrix

In [21]:
#KNN with Cosine Similarity
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(rating_popular_book_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [22]:
# Pickling a python object
file = "book_knn.pkl"
fileobj = open(file, 'wb')
pickle.dump(model_knn, fileobj)
fileobj.close()

In [23]:
file = "book_knn.pkl"
fileobj = open(file, 'rb')
knn = pickle.load(fileobj)
print(knn)
print(type(knn))

NearestNeighbors(algorithm='brute', metric='cosine')
<class 'sklearn.neighbors._unsupervised.NearestNeighbors'>


In [24]:
import requests
def fetch_info(book):
    url = "https://www.googleapis.com/books/v1/volumes?q=title:{}".format(book)
    data = requests.get(url)
    data = data.json()
    item = data['items'][0]
    vol = item['volumeInfo']
    authors = vol['authors']
    publisher = vol['publisher'] if ("publisher" in vol) else ""
    published_date = vol['publishedDate'] if ("published_date" in vol) else ""
    description = vol['description'] if ("description" in vol) else ""
    poster = vol['imageLinks']['thumbnail'] if ("imageLinks" in vol) else ""
    #return description, poster
    return authors, publisher, published_date, description, poster

In [25]:
def getRecommendations(book):
    pd_pvt = rating_popular_book_pivot[(rating_popular_book_pivot.index.get_level_values('book_title') == book)]
    distance, indice = knn.kneighbors(pd_pvt.values, n_neighbors = 6)
    for i in range(0, len(distance.flatten())):
        if i != 0:
            b = rating_popular_book_pivot.index[indice.flatten()[i]]
            print(b)

In [26]:
def getRecommendationsWithDetails(book):
    recommendations = []
    title = []
    authors = []
    publisher = []
    published_date = []
    description = []
    poster = []
    pd_pvt = rating_popular_book_pivot[(rating_popular_book_pivot.index.get_level_values('book_title') == book)]
    distance, indice = knn.kneighbors(pd_pvt.values, n_neighbors = 6)
    for i in range(0, len(distance.flatten())):
        if i != 0:
            b = rating_popular_book_pivot.index[indice.flatten()[i]]
            a, p, pd, d, po = fetch_info(b)
            title.append(b)
            authors.append(a)
            publisher.append(p)
            published_date.append(pd)
            description.append(d)
            poster.append(po)
    for x in range(5):
        d = {}
        d['title'] = title[x]
        d['authors'] = authors[x]
        d['publisher'] = publisher[x]
        d['published_date'] = published_date[x]
        d['description'] = description[x]
        d['poster'] = poster[x]
        recommendations.append(d)
    return recommendations

In [27]:
getRecommendations("Harry Potter and the Chamber of Secrets (Book 2)")

Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Order of the Phoenix (Book 5)
The Fellowship of the Ring (The Lord of the Rings, Part 1)


In [28]:
getRecommendationsWithDetails("Harry Potter and the Chamber of Secrets (Book 2)")

[{'title': 'Harry Potter and the Prisoner of Azkaban (Book 3)',
  'authors': ['J. K. Rowling'],
  'publisher': 'Bloomsbury Publishing',
  'published_date': '',
  'description': "When the Knight Bus crashes through the darkness and screeches to a halt in front of him, it's the start of another far from ordinary year at Hogwarts for Harry Potter. Sirius Black, escaped mass-murderer and follower of Lord Voldemort, is on the run - and they say he is coming after Harry. In his first ever Divination class, Professor Trelawney sees an omen of death in Harry's tea leaves . But perhaps most terrifying of all are the Dementors patrolling the school grounds, with their soul-sucking kiss. These new editions of the classic and internationally bestselling, multi-award-winning series feature instantly pick-up-able new jackets by Jonny Duddle, with huge child appeal, to bring Harry Potter to the next generation of readers. It's time to PASS THE MAGIC ON .",
  'poster': 'http://books.google.com/books/c